In [24]:
#import os
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"

## Local RAG system using LangChain, Ollama and Llama 3.1

#### 1. Import the LLM and the embedding model

In [1]:
#from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3.1"
#model = ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

#### 2. Load the PDF document and split it into "chunks"

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/cloudcomp.pdf")
pages = loader.load_and_split()
pages

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)


[Document(metadata={'source': 'docs/cloudcomp.pdf', 'page': 0}, page_content='What is cloud computing  Cloud computing is the delivery of computing services over the internet. Computing services include common IT infrastructure such as virtual machines, storage, databases, and networking. Cloud services also expand the traditional IT offerings to include things like Internet of Things (IoT), machine learning (ML), and artificial intelligence (AI).  Because cloud computing uses the internet to deliver these services, it doesn’t have to be constrained by physical infrastructure the same way that a traditional datacenter is. That means if you need to increase your IT infrastructure rapidly, you don’t have to wait to build a new datacenter—you can use the cloud to rapidly expand your IT footprint.  Describe the shared responsibility model  Start with a traditional corporate datacenter. The company is responsible for maintaining the physical space, ensuring security, and maintaining or repl

#### 3. Create a prompt template

In [3]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



#### 4. Build a simple chain to test the model

In [4]:
chain = prompt | model 

In [5]:
chain.invoke(
    {
        "context": "The name I was given is Ivan",
        "question": "What is my name?"
    }
) 

'Your name is Ivan.'

#### 5. Create a local vector store (database) to store the embedded docs.

In [6]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

/Users/ivanlee/llama-rag-demo/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


#### 6. Build a chain

The chain first takes the question and sends it to the retreiver in order to get the context from our documents. Then the prompt is sent to the model.

In [7]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    } 
    | prompt
    | model 
)

#### 7. Test the model

In [8]:
questions = [
    "What is cloud computing?",
    "What are the main cloud models?",
    "What is the pricing model of cloud computing?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is cloud computing?
Answer: Cloud computing is the delivery of computing services over the internet by using a pay-as-you-go pricing model. It allows users to rent compute power and storage from someone else’s datacenter, and enables them to quickly solve their toughest business challenges and bring cutting-edge solutions to their users. Computing services include common IT infrastructure such as virtual machines, storage, databases, and networking, as well as expanding the traditional IT offerings to include things like Internet of Things (IoT), machine learning (ML), and artificial intelligence (AI).

Question: What are the main cloud models?
Answer: The three main cloud models are:

1. Private
2. Public
3. Hybrid

Question: What is the pricing model of cloud computing?
Answer: The pricing model of cloud computing is pay-as-you-go, where you typically pay only for the cloud services you use, with no upfront costs. This means that if you don't use any IT resources one m